In [1]:
include("../../src/struct_data.jl")
include("../../src/neighbor.jl")
include("../../src/forces/forces.jl")

cu_force

In [2]:
@time Model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 100000.0,
        dt    = 0.5,
        nₖₙₙ  = 50000,
        nₛₐᵥₑ = 1
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input  = "../../data/init/Sphere"
    ),
    OutputModel(
        name_output = "Test_1",
        path_output = ""
    )
)

  0.000000 seconds (4 allocations: 144 bytes)


ModelSet(TimeModel(100000.0, 0.5, 50000, 1), InputModel(0.8, "../../data/init/Sphere"), OutputModel("Test_1", ""))

In [3]:
Par1, Par2 = (0.5,2.0,3.3), 0.05

FusionAGG = nothing
# @time @start_agg FusionAGG = FusionAggregate(
@time FusionAGG = FusionAggregate(
    [
        AggType(
            "HEK_1", 
            InteractionPar(
                Cubic(Par1...), 
                ContractilePar(Par2)
            ),
            Float64.(readdlm("../../data/init/Sphere/9.0.xyz")[3:end,2:end]) |> cu
        )
    ], 
    Model
)

  4.368155 seconds (13.28 M allocations: 1.381 GiB, 4.11% gc time, 4.44% compilation time)


Aggregate(AggType[AggType("HEK_1", InteractionPar(Cubic{Float64}(0.5, 2.0, 3.3), ContractilePar(0.05)), 8.976667f0, Float32[-1.5 -4.62 -7.35; 0.5 -4.62 -7.35; … ; -1.5 4.62 7.35; 0.5 4.62 7.35], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})], AggIndex([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2], ["HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1"  …  "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1"]), Float32[-10.476667 -4.62 -7.35; -8.476667 -4.62 -7.35; … ; 7.4766674 4.62 7.35; 9.476667 4.62 7.35], AggGeometry(Float32[8.976667, 8.976667, 8.976667, 8.976667, 8.976667, 8.976667, 8.976667, 8.976667, 8.976667, 8.976667  …  8.976667, 8.976667, 8.976667, 8.976667, 8.976667, 8.976667, 8.976667, 8.976667, 8.976667, 8.976667], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), AggSimulation(AggParameter(Cubic{CuArray{Fl

In [4]:
points = FusionAGG.Position
idx    = FusionAGG.Simulation.Neighbor.idx
r_max  = FusionAGG.Simulation.Parameter.Force.rₘₐₓ

# Calculating Distance Matrix
threads=(8,8)
@cuda(
    threads=threads,
    blocks=cld.(size(points,1),threads),
    dist_kernel!(idx, points,r_max)
)

display(idx)

1076×1076 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
 1  1  1  1   0   0   0   0   0   0  …     0     0     0     0     0     0
 2  2  0  2   2   0   0   0   0   0        0     0     0     0     0     0
 3  0  3  3   0   0   3   3   0   0        0     0     0     0     0     0
 4  4  4  4   4   0   0   4   4   0        0     0     0     0     0     0
 0  5  0  5   5   5   0   0   5   5        0     0     0     0     0     0
 0  0  0  0   6   6   0   0   0   6  …     0     0     0     0     0     0
 0  0  7  0   0   0   7   7   0   0        0     0     0     0     0     0
 0  0  8  8   0   0   8   8   8   0        0     0     0     0     0     0
 0  0  0  9   9   0   0   9   9   9        0     0     0     0     0     0
 0  0  0  0  10  10   0   0  10  10        0     0     0     0     0     0
 0  0  0  0   0  11   0   0   0  11  …     0     0     0     0     0     0
 0  0  0  0   0   0  12   0   0   0        0     0     0     0     0     0
 0  0  0  0   0   0  13  13   0   0        0    

In [5]:
idx_red = FusionAGG.Simulation.Neighbor.idx_red
idx_sum = FusionAGG.Simulation.Neighbor.idx_sum

# Reducing Distance Matrix to Nearest Neighbors
threads = 64
@cuda(
    threads=threads,
    blocks=cld.(size(points,1),threads),
    reduce_kernel!(idx,idx_red,idx_sum)
)

display(idx_red)
display(idx_sum)

21×1076 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
  1   1   1   1   2   5   3   3   4  …   991   992   993   994   998   999
  2   2   3   2   4   6   7   4   5     1031  1032  1033  1034  1038  1039
  3   4   4   3   5  10   8   7   8     1037  1038  1039  1040  1044  1045
  4   5   7   4   6  11  12   8   9     1038  1039  1040  1041  1045  1046
 25  26   8   5   9  32  13   9  10     1043  1044  1045  1046  1049  1050
 26  27  29   8  10  33  35  13  14  …  1044  1045  1046  1047  1050  1051
 27  28  30   9  31  34  36  14  15     1045  1046  1047  1048  1051  1052
 30  31  31  30  32  39  37  36  37     1066  1067  1068  1069  1071  1072
 31  32  36  31  33  40  43  37  38     1067  1068  1069  1070  1072  1073
 37  38  37  32  38  47  44  38  39     1071  1071  1072  1073  1075  1075
 77  78  44  37  39  87  50  44  45  …  1072  1072  1073  1074  1076  1076
  0   0  84  38  46   0  91  45  46     1075  1073  1074     0     0     0
  0   0   0  45  86   0   0  51  52        0  1075

1×1076 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
 11  11  12  14  13  11  12  14  14  …  14  14  11  12  14  13  11  11  11

In [6]:
force = FusionAGG.Simulation.Force.F
ParForce = FusionAGG.Simulation.Parameter.Force
ParCont = FusionAGG.Simulation.Parameter.Contractile.fₚ
outline = FusionAGG.Geometry.outline

tₛᵢₘ  = Model.Time.tₛᵢₘ
nₛₐᵥₑ = Model.Time.nₛₐᵥₑ
nₖₙₙ  = Model.Time.nₖₙₙ
dt = Model.Time.dt

while true
    global points_ant
    
    points_ant = copy(points)
    # println(any(isnan.(points_ant)) & any(.!isfinite.(points_ant))) 
    # display(points_ant)

    threads=(16,3)
    @cuda(
        threads=threads,
        blocks=(cld.(size(points,1)+1,threads[1]),1),
        sum_force!(idx_red, points, force, ParForce, dt)
    )

    # println(any(isnan.(points)) & any(.!isfinite.(points)))
    # display(points)
    if any(isnan.(points)) & any(.!isfinite.(points)) == true
        break
    end
end

println(any(isnan.(points_ant)) & any(.!isfinite.(points_ant))) 
display(points_ant)

println(any(isnan.(points)) & any(.!isfinite.(points)))
display(points)

false


1076×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -10.5173   -4.32582   -7.01111
  -8.49381  -4.4636    -6.98038
 -11.534    -2.79309   -7.17613
  -9.49771  -2.7593    -7.10447
  -7.50113  -2.70434   -7.05847
  -5.50596  -2.74958   -7.01596
 -12.5271   -1.01105   -7.00969
 -10.5199   -1.08563   -7.08825
  -8.47777  -1.10253   -7.11651
  -6.42778  -1.072     -7.04929
  -4.40262  -1.03715   -6.98367
 -13.4527    0.81058   -6.8634
 -11.4794    0.760521  -6.87804
   ⋮                   
  12.6311   -0.332354   6.67515
   5.47054   1.09335    6.9151
   7.51468   1.12855    7.05033
   9.54621   1.20321    7.04047
  11.5661    1.30431    6.76553
  13.5798    1.3922     6.79593
   6.43379   2.778      6.87658
   8.44333   2.82893    6.94557
  10.507     2.94945    7.02627
  12.5126    3.06981    6.6772
   7.35868   4.50602    6.75566
   9.36009   4.55749    6.81895

true


1076×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -10.5195   -4.29948   -6.98234
  -8.49737  -4.44531   -6.96142
 -11.5362   -2.78885   -7.1619
  -9.49113  -2.7503    -7.08839
  -7.49577  -2.69046   -7.04503
  -5.49992  -2.73272   -7.00922
 -12.5302   -0.98677   -6.98984
 -10.521    -1.07633   -7.07601
  -8.47261  -1.09246   -7.10542
  -6.41689  -1.0598    -7.03266
  -4.39628  -1.01523   -6.96855
 -13.4473    0.829423  -6.83651
 -11.4774    0.777619  -6.86162
   ⋮                   
  12.6304   -0.307902   6.55836
   5.47917   1.08804    6.87886
   7.52569   1.1218     7.02219
   9.55949   1.20082    7.00041
  11.5768    1.32284    6.68886
  13.606     1.3929     6.79422
   6.44325   2.76681    6.83777
   8.44908   2.82349    6.91399
  10.5096    2.95199    7.01169
  12.5117    3.07873    6.62388
   7.35553   4.49899    6.72217
   9.3568    4.55763    6.78716

In [12]:
sum(points_ant, dims=1)

1×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 215.607  63.9938  52.2656

In [13]:
sum(points_ant, dims=2)

1076×1 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -21.854229
 -19.937786
 -21.5032
 -19.361479
 -17.263945
 -15.271494
 -20.547842
 -18.69378
 -16.69681
 -14.54908
 -12.423439
 -19.505548
 -17.596962
   ⋮
  18.973862
  13.478987
  15.693559
  17.78989
  19.635986
  21.767883
  16.088375
  18.217829
  20.482727
  22.259584
  18.620352
  20.73653

# Later

In [7]:
# force = FusionAGG.Simulation.Force.F
# ParForce = FusionAGG.Simulation.Parameter.Force
# ParCont = FusionAGG.Simulation.Parameter.Contractile.fₚ
# outline = FusionAGG.Geometry.outline

# tₛᵢₘ  = 100.0
# nₛₐᵥₑ = Model.Time.nₛₐᵥₑ
# nₖₙₙ  = Model.Time.nₖₙₙ
# dt = Model.Time.dt

# # Calculating position of every cell on the fusion
# run(`rm Test.xyz `)
# using ProgressMeter
# @showprogress "Forces..." for step::Int=0:tₛᵢₘ/dt

#     # Saving data in a given time (nₛₐᵥₑ)
#     # if (step % Int(tₛᵢₘ/nₛₐᵥₑ)) == 1
#         open("Test.xyz", "a") do f
#             write(f, "$(size(points, 1))\n")
#             write(f, "t=$(step*dt)\n")
#             writedlm(f,hcat(outline,Matrix(points)), ' ')
#         end
#     # end

#     # # Calculating the kNN for the aggregate
#     # t_nₖₙₙ = step % nₖₙₙ+1
#     # if t_nₖₙₙ == 1
        
#         # # Calculating Distance Matrix
#         # threads=(8,8)
#         # @cuda(
#         #     threads=threads,
#         #     blocks=cld.(size(points,1),threads),
#         #     dist_kernel!(idx, points,r_max)
#         # )

#         # # Reducing Distance Matrix to Nearest Neighbors
#         # threads = 64
#         # @cuda(
#         #     threads=threads,
#         #     blocks=cld.(size(points,1),threads),
#         #     reduce_kernel!(idx,idx_red,idx_sum)
#         # )

#     # #     # Finding index contractile
#     # #     # Finding index contractile
#     # #     threads = (8,8)
#     # #     blocks  =cld.(size(points,1),threads)
#     # #     @cuda threads=threads blocks=blocks index_contractile!(idx_contractile,idx_sum,idx_red)
    
#     # end

#     # Compute the forces between each pair of particles in `agg` and their displacement.
#     threads=(16,3)
#     @cuda(
#         threads=threads,
#         blocks=(cld.(size(points,1)+1,threads[1]),1),
#         sum_force!(idx_red, points, force, ParForce, dt)
#     )
# end
# display(points)

In [8]:
# idx_contractile = FusionAGG.Simulation.Neighbor.idx_cont

# # Finding index contractile
# threads = (8,8)
# @cuda(
#     threads=threads,
#     blocks=cld.(size(points,1),threads),
#     index_contractile!(idx_contractile,idx_sum,idx_red)
# )

# display(idx_contractile)